In [86]:
!pip install Sastrawi


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [87]:
!pip install nltk


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [88]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\asus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [89]:
import pandas as pd
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from wordcloud import WordCloud
import csv

from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, LSTM

In [90]:
data_tw = pd.read_csv('KonserColdplay.csv',encoding = 'unicode_escape', delimiter=";")
data = data_tw
data.head()

,Text,Id
0,'Awalnya (Korban Penipuan Tiket Konser Coldpla...,1.660908e+18
1,Pasangan suami istri pelaku penipuan tiket kon...,1.660593e+18
2,{Thread}\n\nPolisi menerima 60 laporan dari ko...,1.660850e+18
3,RT @mazzini_gsp: 'Awalnya (Korban Penipuan Tik...,1.660962e+18
4,RT @mazzini_gsp: 'Awalnya (Korban Penipuan Tik...,1.660962e+18


In [93]:
print(data.columns)

Index(['Text', 'Id'], dtype='object')


In [91]:
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # remove mentions
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # remove hashtag
    text = re.sub(r'RT[\s]', '', text) # remove RT
    text = re.sub(r"http\S+", '', text) # remove link
    text = re.sub(r'[0-9]+', '', text) # remove numbers
    text = text.replace('\n', ' ') # replace new line into space
    text = text.translate(str.maketrans('', '', string.punctuation)) # remove all punctuations
    text = text.strip(' ') # remove characters space from both left and right text
    return text

def casefoldingText(text): # Converting all the characters in a text into lower case
    text = text.lower() 
    return text

def tokenizingText(text): # Tokenizing or splitting a string, text into a list of tokens
    text = word_tokenize(text) 
    return text

def filteringText(text): # Remove stopwors in a text
    listStopwords = set(stopwords.words('indonesian'))
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered 
    return text

def stemmingText(text): # Reducing a word to its word stem that affixes to suffixes and prefixes or to the roots of words
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    text = [stemmer.stem(word) for word in text]
    return text

def toSentence(list_words): # Convert list of words into sentence
    sentence = ' '.join(word for word in list_words)
    return sentence

In [95]:
data['text_clean'] = ""
data['text_clean'] = data['Text'].apply(cleaningText)
data['text_clean'] = data['text_clean'].apply(casefoldingText)
data.drop(['Text'], axis = 1, inplace = True)

data['text_preprocessed'] = data['text_clean'].apply(tokenizingText)
data['text_preprocessed'] = data['text_preprocessed'].apply(filteringText)
data['text_preprocessed'] = data['text_preprocessed'].apply(stemmingText)

# Membuang duplikat tweets/spam tweets
data.drop_duplicates(subset = 'text_clean', inplace = True)

In [97]:
data.head()

,Id,text_clean,text_preprocessed
0,1.660908e+18,awalnya korban penipuan tiket konser coldplay ...,"[korban, tipu, tiket, konser, coldplay, orang,..."
1,1.660593e+18,pasangan suami istri pelaku penipuan tiket kon...,"[pasang, suami, istri, laku, tipu, tiket, kons..."
2,1.660850e+18,thread polisi menerima laporan dari korban p...,"[thread, polisi, terima, lapor, korban, tipu, ..."
3,1.660962e+18,gsp awalnya korban penipuan tiket konser coldp...,"[gsp, korban, tipu, tiket, konser, coldplay, o..."
5,1.660962e+18,subdit siber dit reskrimum polda metro jaya me...,"[subdit, siber, dit, reskrimum, polda, metro, ..."


In [99]:
data.to_csv('Coldplay_Clean.csv')